In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def lerarquivos(mes):
     #Caminho do arquivo CSV
    url = f"../dados-tratados/pc-{mes:0>2}.csv"

    # Carregando o dataset
    return pd.read_csv(url, sep=";")


In [9]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt


# Faixas de renda e sintomas
faixas_renda = [
    "0 - 100", "101 - 300", "301 - 600", "601 - 800",
    "801 - 1.600", "1.601 - 3.000", "3.001 - 10.000", "10.001 - 50.000"
]

colunas_sintomas = [
    "Febre", "Tosse", "Dor_garganta", "Dificuldade_respirar",
    "Dor_cabeca", "Dor_peito", "Nausea", "Nariz_entupido",
    "Fadiga", "Dor_olhos", "Perda_cheiro", "Dor_muscular"
]

# Loop para os meses (ajuste conforme necessário)
for m in range(5, 7):
    df = lerarquivos(m)

    # Preenchendo NaN na coluna de Faixa_renda e filtrando faixas
    faixa_renda_coluna = "Faixa_renda"
    df[faixa_renda_coluna] = df[faixa_renda_coluna].fillna("NaN")  # Evitar erro com NaN
    df = df[df[faixa_renda_coluna].isin(faixas_renda)]

    if "Diarreia" in df.columns:
        colunas_sintomas.append("Diarreia")
        
    # Criando o grafo
    G = nx.Graph()

    # Adicionando os nós de faixa de renda à esquerda e sintomas à direita
    G.add_nodes_from(faixas_renda, bipartite=0)  # Faixas de renda à esquerda
    G.add_nodes_from(colunas_sintomas, bipartite=1)  # Sintomas à direita

    # Contagem de pessoas por faixa de renda
    total_por_faixa = df[faixa_renda_coluna].value_counts().to_dict()

    # Calculando as arestas entre faixa de renda e sintomas
    pesos = {}

    for _, row in df.iterrows():
        faixa_renda = row[faixa_renda_coluna]
        for sintoma in colunas_sintomas:
            if row[sintoma] == "Sim":  # Considerando "Sim" como presença do sintoma
                peso = 1  # A cada "Sim", adiciona 1

                # Se a aresta já existe, soma o peso
                if (faixa_renda, sintoma) in pesos:
                    pesos[(faixa_renda, sintoma)] += peso
                else:
                    pesos[(faixa_renda, sintoma)] = peso

    # Calculando os pesos percentuais das arestas em relação à quantidade de pessoas na faixa de renda
    pesos_percentuais = {
        (u, v): (peso / total_por_faixa[u]) * 100 for (u, v), peso in pesos.items()
    }

    # Adicionando as arestas ao grafo com os pesos em percentual
    for (u, v), peso in pesos_percentuais.items():
        G.add_edge(u, v, weight=peso)

    # Definindo a posição dos nós (faixa de renda à esquerda, sintomas à direita)
    pos = {}

    # Posicionando as faixas de renda à esquerda com espaçamento uniforme
    faixa_renda_y = np.linspace(0, 12, len(faixas_renda))  # Espaçamento uniforme ao longo do eixo Y
    for i, (faixa, y) in enumerate(zip(faixas_renda, faixa_renda_y)):
        pos[faixa] = (0, y)  # Faixas de renda na posição fixa à esquerda

    # Posicionando os sintomas à direita com espaçamento uniforme
    sintoma_y = np.linspace(0, 12, len(colunas_sintomas))  # Espaçamento uniforme ao longo do eixo Y
    for i, (sintoma, y) in enumerate(zip(colunas_sintomas, sintoma_y)):
        pos[sintoma] = (1, y)  # Sintomas na posição fixa à direita

    # Extrair os pesos das arestas para ajustar a espessura
    edges = G.edges(data=True)
    max_peso = max(pesos_percentuais.values()) if pesos_percentuais else 1  # Evitar divisão por zero
    weights = [edata["weight"] / max_peso * 3 for _, _, edata in edges]  # Ajuste de espessura

    # Rótulos nas arestas com as porcentagens
    edge_labels = {(u, v): f"{d['weight']:.2f}%" for u, v, d in G.edges(data=True)}

    # Plotando o grafo
    plt.figure(figsize=(14, 8))
    nx.draw(G, pos, with_labels=True, node_size=2000, node_color="lightblue", edge_color="gray", width=weights, font_size=9)
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

    plt.title(f"Grafo: Faixa de Renda X Sintomas (Pesos em Percentual) Mês: {m:02}")
    plt.show()
